In [1]:
# set parameters
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import dmg5e
import estats5e
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True

COLORS = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

ABILITY_MOD_COLUMNS = ['Str Mod','Dex Mod','Con Mod','Int Mod','Wis Mod','Cha Mod',]
SAVE_BONUS_COLUMNS = ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save',]
CONDITION_COLUMNS = ['Blinded','Charmed','Deafened','Exhaustion','Frightened','Grappled','Incapacitated',
    'Invisible','Paralyzed','Petrified','Poisoned','Prone','Restrained','Stunned','Unconscious',]
DAMAGE_COLUMNS = ['Bludgeoning','Piercing','Slashing','Acid','Cold','Fire','Force','Lightning','Necrotic','Poison','Psychic','Radiant','Thunder',]

In [2]:
TFB_MONSTER_DEFAULTS = {
    'CR': [0, 1/8, 1/4, 1/2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 
        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'XP': [2, 15, 38, 70, 152, 330, 583, 980, 1406, 2104, 2715, 3640, 4593, 
        5899, 7006, 8666, 10189, 12361, 14080, 16748, 19016, 21808, 24575, 29344, 
        35879, 44877, 55101, 66811, 78655, 92811, 108626, 126102, 143173, 165404],
    'PB': [2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 
        5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9],
    'AC': [13, 13, 13, 13, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 17, 18, 18, 
        18, 19, 19, 19, 19, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 23],
    'SB': [0, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 7, 
        8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14],
    'HP': [4.0, 9.5, 15.0, 21.0, 32.0, 49.0, 66.0, 83.0, 100.0, 117.0, 134.0, 
        151.0, 168.0, 184.5, 201.0, 218.0, 235.0, 252.0, 269.0, 286.0, 303.0, 320.0, 
        337.0, 361.5, 402.0, 450.5, 499.0, 547.5, 596.0, 644.5, 692.5, 741.0, 789.5, 838.0],
    'AB': [5, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 
        12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19],
    'DC': [13, 13, 13, 13, 13, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 
        18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27],
    'DPR': [1.5, 5.0, 8.0, 10.5, 14.5, 20.0, 25.5, 31.5, 37.5, 43.5, 49.0, 54.5, 
        60.5, 66.5, 72.5, 78.0, 83.5, 89.5, 95.5, 101.5, 107.0, 112.5, 118.5, 126.0, 
        136.5, 148.0, 159.5, 171.5, 183.0, 194.5, 206.5, 218.5, 230.0, 241.5],
}

def _find_nearest_loc(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def monster_default_stats(cr):
    """Returns default monster stats.
    cr -- challenge rating
    """
    id = _find_nearest_loc(TFB_MONSTER_DEFAULTS['CR'], cr)
    return {
        'CR': TFB_MONSTER_DEFAULTS['CR'][id],
        'XP': TFB_MONSTER_DEFAULTS['XP'][id],
        'PB': TFB_MONSTER_DEFAULTS['PB'][id],
        'AC': TFB_MONSTER_DEFAULTS['AC'][id],
        'SB': TFB_MONSTER_DEFAULTS['SB'][id],
        'HP': TFB_MONSTER_DEFAULTS['HP'][id],
        'AB': TFB_MONSTER_DEFAULTS['AB'][id],
        'DC': TFB_MONSTER_DEFAULTS['DC'][id],
        'DPR': TFB_MONSTER_DEFAULTS['DPR'][id],
    }

def count_save_proficiencies(row):
    n = 0
    for ab in ['Str','Dex','Con','Int','Wis','Cha']:
        if row[f'{ab} Save'] > row[f'{ab} Mod']:
            n += 1
    return n

def average_ability_modifier(row):
    v  = row[f'Str Mod'] + row[f'Dex Mod'] + row[f'Con Mod'] 
    v += row[f'Int Mod'] + row[f'Wis Mod'] + row[f'Cha Mod']
    return v/6

def initiative_adjustment(row):
    return 1 + (0.5/2.5)*(row['Initiative']/20)

In [4]:
dfT = pd.DataFrame(TFB_MONSTER_DEFAULTS)
dfT['adj AC'] = dfT['AC']
dfT['adj SB'] = dfT['SB']
dfT['adj HP'] = dfT['HP']
dfT['adj AB'] = dfT['AB']
dfT['adj DC'] = dfT['DC']
dfT['adj DPR'] = dfT['DPR']
dfT['HP multiplier'] = dfT['adj HP']/dfT['HP']
dfT['eAC'] = (dfT['AC'] + dfT['SB'] + 14)/2
dfT['eHP']  = dfT.apply(lambda row: estats5e.effHP(row['adj HP'], row['eAC']), axis=1)
dfT['eDPR'] = dfT.apply(lambda row: estats5e.effDPR(row['adj DPR'], row['adj AB']), axis=1)
dfT['eXP']  = dfT.apply(lambda row: estats5e.effXP(row['adj HP'], row['eAC'], row['adj DPR'], row['adj AB']), axis=1)
dfT['adj AC Delta'] = dfT['adj AC'] - dfT['adj AC']
dfT['adj SB Delta'] = dfT['adj SB'] - dfT['adj SB']
dfT['adj AB Delta'] = dfT['adj AB'] - dfT['adj AB']
dfT['HP Ratio']  = dfT['HP']/dfT['HP']
dfT['adj HP Ratio']  = dfT['adj HP']/dfT['adj HP']
dfT['adj DPR Ratio']  = dfT['adj DPR']/dfT['adj DPR']
dfT['eHP Ratio']  = dfT['eHP']/dfT['eHP']
dfT['eDPR Ratio']  = dfT['eDPR']/dfT['eDPR']
dfT['eXP Ratio']  = dfT['eXP']/dfT['XP']

In [5]:
name_decoder = {
    # 2014 name --> 2025 equivalent
    'Aarakocra': 'Aarakocra Skirmisher',
    'Acolyte': 'Priest Acolyte',
    'Adult Blue Dracolich': 'Dracolich',
    'Androsphinx': 'Sphinx of Valor',
    'Azer': 'Azer Sentinel',
    'Bugbear': 'Bugbear Warrior',
    'Bullywug': 'Bullywug Warrior',
    'Centaur': 'Centaur Trooper',
    'Cult Fanatic': 'Cultist Fanatic',
    'Cyclops': 'Cyclops Sentry',
    'Deep Gnome': 'Scout',
    'Drow': 'Priest Acolyte',
    'Drow Elite Warrior': 'Gladiator',
    'Drow Mage': 'Bandit Deceiver',
    'Drow Priestess of Lolth': 'Fiend Cultist',
    'Duergar': 'Spy',
    'Duodrone': 'Modron Duodrone',
    'Faerie Dragon (Older)': 'Faerie Dragon Adult',
    'Faerie Dragon (Younger)': 'Faerie Dragon Youth',
    'Fire Snake': 'Salamander Fire Snake',
    'Flying Sword': 'Animated Flying Sword',
    'Gas Spore': 'Gas Spore Fungus',
    'Giant Poisonous Snake': 'Giant Venomous Snake',
    'Gnoll': 'Gnoll Warrior',
    'Goblin': 'Goblin Warrior',
    'Grick Alpha': 'Grick Ancient',
    'Gynosphinx': 'Sphinx of Lore',
    'Half-Ogre': 'Ogrillon Ogre',
    'Half-Red Dragon Veteran': 'Half-Dragon',
    'Hobgoblin': 'Hobgoblin Warrior',
    'Kobold': 'Kobold Warrior',
    'Lizardfolk': 'Scout',
    'Lizardfolk Shaman': 'Lizardfolk Geomancer',
    'Lizard King/Queen': 'Lizardfolk Sovereign',
    'Merfolk': 'Merfolk Skirmisher',
    'Minotaur': 'Minotaur of Baphomet',
    'Monodrone': 'Modron Monodrone',
    'Orc': 'Tough',
    'Orc Eye of Gruumsh': 'Cultist Fanatic',
    'Orc War Chief': 'Tough Boss',
    'Orog': 'Berserker',
    'Pentadrone': 'Modron Pentadrone',
    'Poisonous Snake': 'Venomous Snake',
    'Quadrone': 'Modron Quadrone',
    'Quaggoth Spore Servant': 'Myconid Spore Servant',
    'Quipper': 'Piranha',
    'Rug of Smothering': 'Animated Rug of Smothering',
    'Sahuagin': 'Sahuagin Warrior',
    'Sahuagin Priestess': 'Sahuagin Priest',
    'Shrieker': 'Shrieker Fungus',
    'Swarm of Poisonous Snakes': 'Swarm of Venomous Snakes',
    'Swarm of Quippers': 'Swarm of Piranhas',
    'Thri-kreen': 'Thri-kreen Marauder',
    'Thug': 'Tough',
    'Tribal Warrior': 'Warrior Infantry',
    'Tridrone': 'Modron Tridrone',
    'Veteran': 'Warrior Veteran',
    "Will-o'-Wisp": "Will-o’-Wisp",
    'Young Red Shadow Dragon': 'Shadow Dragon',
    'Yuan-ti Pureblood': 'Yuan-ti Infiltrator',
}

In [51]:
# load monster data
df1 = pd.read_csv('./monsters-2014.csv')
df2 = pd.read_csv('./monsters-2024.csv')
df0 = pd.concat([df1, df2])

df0['Str Save Prof'] = df0.apply(lambda row: int(row['Str Save'] > row['Str Mod']), axis=1)
df0['Dex Save Prof'] = df0.apply(lambda row: int(row['Dex Save'] > row['Dex Mod']), axis=1)
df0['Con Save Prof'] = df0.apply(lambda row: int(row['Con Save'] > row['Con Mod']), axis=1)
df0['Int Save Prof'] = df0.apply(lambda row: int(row['Int Save'] > row['Int Mod']), axis=1)
df0['Wis Save Prof'] = df0.apply(lambda row: int(row['Wis Save'] > row['Wis Mod']), axis=1)
df0['Cha Save Prof'] = df0.apply(lambda row: int(row['Cha Save'] > row['Cha Mod']), axis=1)

df0['Save Proficiencies'] = df0.apply(lambda row: count_save_proficiencies(row), axis=1)

df0['DC'] = df0['Save DC']
df0['adj DC'] = df0['DC']
df0['AM'] = df0.apply(lambda row: average_ability_modifier(row), axis=1)
df0['HP multiplier'] = df0['adj HP']/df0['HP']

df0['eAC']  = (df0['adj AC'] + df0['adj SB'] + 14)/2
df0['eHP']  = df0.apply(lambda row: estats5e.effHP(row['adj HP'], row['eAC']), axis=1)
df0['eDPR'] = df0.apply(lambda row: estats5e.effDPR(row['adj DPR'], row['adj AB']), axis=1)
df0['eXP']  = df0.apply(lambda row: estats5e.effXP(row['adj HP'], row['eAC'], row['adj DPR'], row['adj AB']), axis=1)
#df0['eXP']  = df0.apply(lambda row: estats5e.effXP(row['adj HP'], row['eAC'], row['adj DPR'], row['adj AB'])*initiative_adjustment(row), axis=1)
df0['eCR']  = df0.apply(lambda row: np.round(np.interp(row['eXP'], dfT['eXP'],  dfT['CR'], left=0, right=30), 0), axis=1)
df0['eCR Delta']  = df0['eCR'] - df0['CR']


#df0['XP'] = df0.apply(lambda row: dmg5e.monster_default_stats(row['CR'])['XP'], axis=1)
df0['XP'] = df0.apply(lambda row: monster_default_stats(row['CR'])['XP'], axis=1)
df0['Monster Alias'] = df0.apply(lambda row: name_decoder.get(row['Monster'], row['Monster']), axis=1)

dfB = dfT.set_index('CR')
df0['adj AB Delta'] = df0.apply(lambda row: row['adj AB'] - dfB.loc[row['CR']]['adj AB'], axis=1)
df0['adj AC Delta'] = df0.apply(lambda row: row['adj AC'] - dfB.loc[row['CR']]['adj AC'], axis=1)
df0['adj SB Delta'] = df0.apply(lambda row: row['adj SB'] - dfB.loc[row['CR']]['adj SB'], axis=1)
df0['HP Ratio'] = df0.apply(lambda row: row['HP']/dfB.loc[row['CR']]['HP'], axis=1)
df0['adj HP Ratio'] = df0.apply(lambda row: row['adj HP']/dfB.loc[row['CR']]['adj HP'], axis=1)
df0['adj DPR Ratio'] = df0.apply(lambda row: row['adj DPR']/dfB.loc[row['CR']]['adj DPR'], axis=1)
df0['eHP Ratio'] = df0.apply(lambda row: row['eHP']/dfB.loc[row['CR']]['eHP'], axis=1)
df0['eDPR Ratio'] = df0.apply(lambda row: row['eDPR']/dfB.loc[row['CR']]['eDPR'], axis=1)
df0['eXP Ratio'] = df0.apply(lambda row: row['eXP']/dfB.loc[row['CR']]['eXP'], axis=1)
df0 = df0[df0['CR'].between(1,30)]
df0

,Monster,Book,Page,Type,Size,Category,Legendary,CR,PB,XP,...,Monster Alias,adj AB Delta,adj AC Delta,adj SB Delta,HP Ratio,adj HP Ratio,adj DPR Ratio,eHP Ratio,eDPR Ratio,eXP Ratio
1,Aboleth,MM,13.0,aberration,large,legendary,LL,10.0,4,5899,...,Aboleth,0.0,0.0,-0.3,0.731707,0.731707,1.473684,0.725931,1.473684,1.071836
2,Abominable Yeti,MM,306.0,monstrosity,huge,generic,N,9.0,4,4593,...,Abominable Yeti,3.0,-2.0,-2.0,0.818452,1.056548,1.190083,0.942326,1.388430,1.310884
4,Adult Black Dragon,MM,88.0,dragon,huge,legendary,L,14.0,5,12361,...,Adult Black Dragon,0.0,1.0,-0.5,0.775794,1.132937,1.225326,1.146753,1.225326,1.400394
5,Adult Blue Dracolich,MM,84.0,undead,huge,legendary,L,17.0,6,19016,...,Dracolich,1.0,0.0,3.7,0.742574,1.039604,1.177570,1.129058,1.231096,1.338600
6,Adult Blue Dragon,MM,91.0,dragon,huge,legendary,L,16.0,5,16748,...,Adult Blue Dragon,0.0,0.0,0.0,0.786713,1.101399,1.241379,1.101399,1.241379,1.367253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Yuan-ti Abomination,MM (2024),345.0,Monstrosity,Large,generic,N,7.0,3,2715,...,Yuan-ti Abomination,0.0,-1.0,4.2,0.951493,0.951493,1.053061,1.038486,1.053061,1.076546
535,Yuan-ti Infiltrator,MM (2024),342.0,Monstrosity,Medium,generic,N,1.0,2,152,...,Yuan-ti Infiltrator,-1.0,-3.0,4.8,1.265625,1.265625,0.620690,1.341563,0.579310,0.780939
536,Yuan-ti Malison (Type 1),MM (2024),343.0,Monstrosity,Medium,generic,N,3.0,2,583,...,Yuan-ti Malison (Type 1),0.0,-3.0,5.0,1.000000,1.000000,0.980392,1.062500,0.980392,1.034858
537,Yuan-ti Malison (Type 2),MM (2024),343.0,Monstrosity,Medium,generic,N,3.0,2,583,...,Yuan-ti Malison (Type 2),0.0,-3.0,5.0,1.000000,1.000000,1.137255,1.062500,1.137255,1.200436


In [76]:
# Fig. 1: Attack bonus vs challenge rating
column = 'adj AB'

groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text=f'attack bonus',
            #range=[0,3],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'AB %{y:.1f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ab-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-ab-vs-cr-small')

In [8]:
# Fig. 2: Save DC vs challenge rating
column = 'DC'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text=f'save difficulty class',
            #range=[0,3],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'DC %{y:.1f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-save-dc-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-save-dc-vs-cr-small')

In [9]:
# Fig. 3: DPR vs challenge rating
column = 'adj DPR'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='damage per round',
            #range=[0,3],
            tick0=0, dtick=50,
            minor=dict(tick0=0, dtick=10),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'DPR %{y:.1f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-dpr-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-dpr-vs-cr-small')

In [170]:
# Fig. 4: DPR vs challenge rating
column = 'adj DPR Ratio'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='normalized damage per round',
            range=[0,2],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
            tickformat='.1f',
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'DPR ratio %{y:.2f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-dpr-ratio-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-dpr-ratio-vs-cr-small')

In [763]:
# Fig. 5: armor class vs challenge rating
column = 'adj AC'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='armor class',
            range=[12,30],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
            tickformat='.0f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'AC %{y:.1f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ac-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-ac-vs-cr-small')

In [11]:
# Fig. 6: save bonus vs challenge rating
column = 'adj SB'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save bonus',
            range=[-2,16],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
            tickformat='.0f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'SB %{y:.1f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sb-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sb-vs-cr-small')

In [12]:
# Fig. 7: number of monsters who gained or lost saving throw proficiencies
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='relative',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters',
            #range=[0,100],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

names = ['Strength','Dexterity','Constitution','Intelligence','Wisdom','Charisma']
stats = [
    'Str Save Prof','Dex Save Prof','Con Save Prof',
    'Int Save Prof','Wis Save Prof','Cha Save Prof',
]

df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP'] + stats, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]
m_count = dfMP.shape[0]

data = {}
for name, stat in zip(names, stats):
    data[name] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) > 0)

fig.add_trace(go.Bar(
    #x=np.array([k[0:3] for k in data.keys()]),
    x=np.array(names),
    y=np.array(list(data.values()))*(1), #(100/m_count),
    name='gained proficiency',
    hovertemplate = ''
        + '%{y:,.0f} gained proficiency'
        + '<extra></extra>'
))

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) < 0)

fig.add_trace(go.Bar(
    #x=np.array([k[0:3] for k in data.keys()]),
    x=np.array(names),
    y=np.array(list(data.values()))*(-1), #(-100/m_count),
    customdata=np.array(list(data.values())),
    name='lost proficiency',
    hovertemplate = ''
        + '%{customdata:,.0f} lost proficiency'
        + '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sp-delta-by-ability-large')
    fig.update_xaxes(tickmode='array', tickvals=names, ticktext=[n[0:3] for n in names])
    tfb.save_fig_html(fig, format='small', name=f'./fig-sp-delta-by-ability-small')

In [22]:
# Fig. 8: save bonus vs challenge rating
columns = ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']
groups = [
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save bonus',
            range=[-2,16],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
            tickformat='.0f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    

    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR','SB','adj SB',*columns]].groupby(['CR']).mean().reset_index()

    for column, color in zip(columns, colors):
        tfb.plot_data_and_fit(
            fig, 
            x=dfG['CR'], 
            y=dfG[column] + dfG['adj SB'] - dfG['SB'],
            name=column,
            legendgroup=column,
            line=dict(color=color),
            print_coefficients=False,
            hovertemplate = f'<b>{name}</b><br>'
                + 'CR %{x:.0f}<br>'
                + column + ' %{y:.1f}'
                + '<extra></extra>'
        )

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-all-sb-vs-cr-large', marker_size=6)
    tfb.save_fig_html(fig, format='small', name=f'./fig-all-sb-vs-cr-small', marker_size=4)

In [14]:
# Fig. 9: hit points vs challenge rating
column = 'adj HP'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='hit points',
            tick0=0, dtick=200,
            minor=dict(tick0=0, dtick=100),
            tickformat='.0f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'HP %{y:.1f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-hp-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-hp-vs-cr-small')

In [15]:
# Fig. 10: hit points vs challenge rating
column = 'adj HP Ratio'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='normalized hit points',
            range=[0,2],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'HP ratio %{y:.2f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-hp-ratio-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-hp-ratio-vs-cr-small')

In [16]:
# Fig. 11: Shows normalized XP vs challenge rating
column = 'eXP Ratio'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='normalized XP',
            range=[0,2.5],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'XP ratio %{y:.2f}'
            + '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')

In [ ]:
# Fig. 12: Shows normalized XP distributions
column = 'eXP Ratio'
groups = [
    {
        'name': 'MM 2014 - normal',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2014 - legendary',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'square',
    },
    {
        'name': 'MM 2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1], 
        'line_color': COLORS[1], 
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1], 
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=65, t=20, pad=4),
        xaxis=dict(
            autorange=True,
            tickmode='array',
            tickvals=['MM 2014 - normal','MM 2014 - legendary','MM 2024 - normal','MM 2024 - legendary'],
            ticktext=['normal','legendary','normal','legendary'],
        ),
        yaxis=dict(
            title_text=f'normalized XP',
            range=[0, 2.5],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
    )
)

for group in groups:
    book = group['Book']
    name = group['name']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Box(
        y=df1[column],
        name=name,
        customdata=df1['Monster'],
        showlegend=False,
        boxpoints='all',
        jitter=0.8,
        whiskerwidth=0.4,
        pointpos=-2,
        marker_size=2,
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        hoveron='points',
        hovertemplate=f'<b>{name}</b><br>'+
            '%{customdata}<br>'+
            'XP ratio %{y:.2f}<br>'+
            '<extra></extra>'
    ))

fig.add_annotation(
    text='MM 2014',
    xref='paper', yref='paper',
    xanchor='center', yanchor='top',
    x=0.3, y=-0.1, 
    showarrow=False,
)
fig.add_annotation(
    text='MM 2024',
    xref='paper', yref='paper',
    xanchor='center', yanchor='top',
    x=0.73, y=-0.1, 
    showarrow=False,
)

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-distribution-large')
    fig.update_traces(marker_size=1)
    tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-distribution-small')

In [ ]:
# Fig. 13: initiative vs challenge rating.
column = 'Initiative'
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='initiative bonus',
            autorange=True,
            #range=[0,3],
            #tick0=0, dtick=50,
            #minor=dict(tick0=0, dtick=10),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'Initiative %{y:.1f}'
            + '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-initiative-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-initiative-vs-cr-small')

# Unused Plots

In [685]:
column = 'adj AB Delta'
groups = [
    {
        'name': '2014 - normal',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0], #'rgba(0,0,250,1)',
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'circle',
    },
    {
        'name': '2014 - legendary',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'square',
    },
    {
        'name': '2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1], #'rgba(250,0,0,1)',
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'circle',
    },
    {
        'name': '2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            #title_text='encounter type',
            autorange=True,
            
        ),
        yaxis=dict(
            title_text=f'{column}',
            autorange=True,
            #range=[0, 2.5],
            #tick0=0, dtick=0.5,
            #minor=dict(tick0=0, dtick=0.1),
        ),
    )
)

for group in groups:
    book = group['Book']
    name = group['name']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Box(
        y=df1[column],
        name=name,
        customdata=df1['Monster'],
        showlegend=False,
        boxpoints='all',
        jitter=0.4,
        whiskerwidth=0.4,
        marker_size=2,
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        hoveron='points',
        hovertemplate=f'<b>{name}</b><br>'+
            '%{customdata}<br>'+
            column + ' %{y:.2f}<br>'+
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [688]:
column = 'adj DPR Ratio'
groups = [
    {
        'name': '2014 - normal',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0], #'rgba(0,0,250,1)',
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'circle',
    },
    {
        'name': '2014 - legendary',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'square',
    },
    {
        'name': '2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1], #'rgba(250,0,0,1)',
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'circle',
    },
    {
        'name': '2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            #title_text='encounter type',
            autorange=True,
            
        ),
        yaxis=dict(
            title_text=f'{column}',
            range=[0, 2],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
    )
)

for group in groups:
    book = group['Book']
    name = group['name']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Box(
        y=df1[column],
        name=name,
        customdata=df1['Monster'],
        showlegend=False,
        boxpoints='all',
        jitter=0.4,
        whiskerwidth=0.4,
        marker_size=2,
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        hoveron='points',
        hovertemplate=f'<b>{name}</b><br>'+
            '%{customdata}<br>'+
            column + ' %{y:.2f}<br>'+
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [692]:
column = 'HP'

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text=f'{column}',
            #range=[0,3],
            tick0=0, dtick=200,
            minor=dict(tick0=0, dtick=100),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
book = 'MM'
df1 = df0[df0['Book'].isin([book])]
dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=dfG[column],
    mode='markers+lines', 
    name=f'{book}',
    hovertemplate = f'<b>{book}</b><br>'
        + 'CR %{x:.0f}<br>'
        + column +' %{y:.1f}'
        + '<extra></extra>'
))

# plot monster data
book = 'MM (2024)'
df1 = df0[df0['Book'].isin([book])]
dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=dfG[column],
    mode='markers+lines', 
    name=f'{book}',
    hovertemplate = f'<b>{book}</b><br>'
        + 'CR %{x:.0f}<br>'
        + column +' %{y:.1f}'
        + '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    line_color='black',
    line_dash='dash',
    hovertemplate = f'<b>{book}</b><br>'
        + 'CR %{x:.0f}<br>'
        + 'DPR %{y:.1f}'
        + '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [694]:
column = 'AC'

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text=f'{column}',
            #range=[0,3],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
book = 'MM'
df1 = df0[df0['Book'].isin([book])]
dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=dfG[column],
    mode='markers+lines', 
    name=f'{book}',
    hovertemplate = f'<b>{book}</b><br>'
        + 'CR %{x:.0f}<br>'
        + column +' %{y:.1f}'
        + '<extra></extra>'
))

# plot monster data
book = 'MM (2024)'
df1 = df0[df0['Book'].isin([book])]
dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=dfG[column],
    mode='markers+lines', 
    name=f'{book}',
    hovertemplate = f'<b>{book}</b><br>'
        + 'CR %{x:.0f}<br>'
        + column +' %{y:.1f}'
        + '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=dfT['CR'], 
    y=dfT[column],
    mode='lines', 
    name=f'baseline',
    line_color='black',
    line_dash='dash',
    hovertemplate = f'<b>{book}</b><br>'
        + 'CR %{x:.0f}<br>'
        + 'DPR %{y:.1f}'
        + '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [696]:
column = 'adj AC Delta'
groups = [
    {
        'name': '2014 - normal',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0], #'rgba(0,0,250,1)',
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'circle',
    },
    {
        'name': '2014 - legendary',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'square',
    },
    {
        'name': '2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1], #'rgba(250,0,0,1)',
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'circle',
    },
    {
        'name': '2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            #title_text='encounter type',
            autorange=True,
            
        ),
        yaxis=dict(
            title_text=f'{column}',
            autorange=True,
            #range=[0, 2.5],
            #tick0=0, dtick=0.5,
            #minor=dict(tick0=0, dtick=0.1),
        ),
    )
)

for group in groups:
    book = group['Book']
    name = group['name']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Box(
        y=df1[column],
        name=name,
        customdata=df1['Monster'],
        showlegend=False,
        boxpoints='all',
        jitter=0.4,
        whiskerwidth=0.4,
        marker_size=2,
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        hoveron='points',
        hovertemplate=f'<b>{name}</b><br>'+
            '%{customdata}<br>'+
            column + ' %{y:.2f}<br>'+
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [ ]:
column = 'Save Proficiencies'

groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N','L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N','L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save proficiencies',
            range=[0,6],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=1),
            tickformat='.0f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    legendary = group['Legendary']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(legendary)]
    dfG = df1[['CR',column]].groupby(['CR']).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['CR'], 
        y=dfG[column],
        mode='markers+lines', 
        name=name,
        marker_color=group['color'],
        marker_symbol=group['symbol'],
        line_color=group['line_color'],
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR %{x:.0f}<br>'
            + 'Proficiencies %{y:.1f}'
            + '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sp-count-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sp-count-vs-cr-small')

In [703]:
column = 'adj SB Delta'
groups = [
    {
        'name': '2014 - normal',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0], #'rgba(0,0,250,1)',
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'circle',
    },
    {
        'name': '2014 - legendary',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'square',
    },
    {
        'name': '2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1], #'rgba(250,0,0,1)',
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'circle',
    },
    {
        'name': '2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            #title_text='encounter type',
            autorange=True,
            
        ),
        yaxis=dict(
            title_text=f'{column}',
            autorange=True,
            #range=[0, 2.5],
            #tick0=0, dtick=0.5,
            #minor=dict(tick0=0, dtick=0.1),
        ),
    )
)

for group in groups:
    book = group['Book']
    name = group['name']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Box(
        y=df1[column],
        name=name,
        customdata=df1['Monster'],
        showlegend=False,
        boxpoints='all',
        jitter=0.4,
        whiskerwidth=0.4,
        marker_size=2,
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        hoveron='points',
        hovertemplate=f'<b>{name}</b><br>'+
            '%{customdata}<br>'+
            column + ' %{y:.2f}<br>'+
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [713]:
column = 'adj HP Ratio'
groups = [
    {
        'name': '2014 - normal',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0], #'rgba(0,0,250,1)',
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'circle',
    },
    {
        'name': '2014 - legendary',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'square',
    },
    {
        'name': '2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1], #'rgba(250,0,0,1)',
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'circle',
    },
    {
        'name': '2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            #title_text='encounter type',
            autorange=True,
            
        ),
        yaxis=dict(
            title_text=f'{column}',
            range=[0, 2.5],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
    )
)

for group in groups:
    book = group['Book']
    name = group['name']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Box(
        y=df1[column],
        name=name,
        customdata=df1['Monster'],
        showlegend=False,
        boxpoints='all',
        jitter=0.4,
        whiskerwidth=0.4,
        marker_size=2,
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        hoveron='points',
        hovertemplate=f'<b>{name}</b><br>'+
            '%{customdata}<br>'+
            column + ' %{y:.2f}<br>'+
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [722]:
column = 'eXP Ratio'
groups = [
    {
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0], #'rgba(0,0,250,1)',
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'circle',
    },
    {
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': 'rgba(0,0,250,0)',
        'line_color': COLORS[0], #'rgba(0,0,250,1)',
        'symbol': 'square',
    },
    {
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1], #'rgba(250,0,0,1)',
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'circle',
    },
    {
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': 'rgba(250,0,0,0)',
        'line_color': COLORS[1], #'rgba(250,0,0,1)',
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text=f'{column}',
            range=[0,2.5],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
            tickformat='.1f',
        ),
        legend=dict(
            xanchor='right', yanchor='bottom',
            x=1.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    book = group['Book']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Scatter(
        x=df1['CR'], 
        y=df1[column],
        mode='markers',
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        name=f'{book}',
        customdata=np.stack((df1['Monster'], df1['CR']), axis=-1),
        hovertemplate = '<b>%{customdata[0]}' + f' - {book}</b><br>'
            + 'CR %{x}<br>'
            + 'eXP Ratio %{y:.3f}'
            + '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [92]:
groups = [
    {
        'name': 'MM 2014 - normal',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0], 
        'line_color': COLORS[0], 
        'symbol': 'circle',
    },
    {
        'name': 'MM 2014 - legendary',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0], 
        'symbol': 'square',
    },
    {
        'name': 'MM 2024 - normal',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024 - legendary',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'square',
    }
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='eDPR Ratio',
            range=[0,3.0],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
            tickformat='.1f',
        ),
        yaxis=dict(
            title_text=f'eHP Ratio',
            range=[0,3.0],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
            tickformat='.1f',
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
for group in groups:
    name = group['name']
    book = group['Book']
    df1 = df0[df0['Book'].isin([book]) & df0['Legendary'].isin(group['Legendary'])]
    fig.add_trace(go.Scatter(
        x=df1['eDPR Ratio'], 
        y=df1['eHP Ratio'],
        mode='markers',
        marker_size=4,
        marker_symbol=group['symbol'],
        marker_color=group['color'],
        marker_line_color=group['line_color'],
        marker_line_width=2,
        name=name,
        customdata=np.stack((df1['Monster'], df1['CR']), axis=-1),
        hovertemplate = '<b>%{customdata[0]}' + f' - {name}</b><br>'
            + 'CR %{customdata[1]}<br>'
            + 'eDPR Ratio %{x:.3f}<br>'
            + 'eHP Ratio %{y:.3f}'
            + '<extra></extra>'
    ))

x = np.linspace(0.2, 3.0, 100)
fig.add_trace(go.Scatter(
    x=x, 
    y=1/x,
    mode='lines', 
    name=f'baseline',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

x = np.linspace(0.2, 3.0, 100)
fig.add_trace(go.Scatter(
    x=x, 
    y=1.6/x,
    mode='lines', 
    name=f'baseline x1.6',
    showlegend=False,
    line_color='black',
    line_dash='dash',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [88]:
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['N'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['N'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='relative',
        xaxis=dict(
            title_text='CR delta',
            range=[-4,4],
        ),
        yaxis=dict(
            title_text='percent of monsters',
            autorange=True,
            #range=[0,100],
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
            tickformat='.0%',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

for group in groups:
    name = group['name']
    book = group['Book']
    dtG = df0[df0['Legendary'].isin(group['Legendary'])].groupby(['Book','eCR Delta']).count()['Monster']
    x = list(dtG[book].keys())
    y = list(dtG[book].values/sum(dtG[book].values))
    fig.add_trace(go.Scatter(
        x=x+[x[-1]+1],
        y=list(dtG[book].values/sum(dtG[book].values)) + [0],
        name=name,
        line_shape='hvh',
        fill='tozeroy',
        hoveron='points',
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR delta %{x:.0f}<br>'
            + 'Monsters %{y:.1%}'
            + '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-sp-delta-by-ability-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-sp-delta-by-ability-small')
    pass

In [ ]:
groups = [
    {
        'name': 'MM 2014',
        'Book': 'MM',
        'Legendary': ['L','LL'],
        'color': COLORS[0],
        'line_color': COLORS[0],
        'symbol': 'circle',
    },
    {
        'name': 'MM 2024',
        'Book': 'MM (2024)',
        'Legendary': ['L','LL'],
        'color': COLORS[1],
        'line_color': COLORS[1],
        'symbol': 'circle',
    },
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='relative',
        xaxis=dict(
            title_text='CR delta',
            range=[-4,6],
        ),
        yaxis=dict(
            title_text='percent of monsters',
            autorange=True,
            #range=[0,100],
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
            tickformat='.0%',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

for group in groups:
    name = group['name']
    book = group['Book']
    dtG = df0[df0['Legendary'].isin(group['Legendary'])].groupby(['Book','eCR Delta']).count()['Monster']
    x = list(dtG[book].keys())
    y = list(dtG[book].values/sum(dtG[book].values))
    fig.add_trace(go.Scatter(
        x=[min(x)-1] + x + [max(x)+1],
        y=[0] + list(dtG[book].values/sum(dtG[book].values)) + [0],
        name=name,
        line_shape='hvh',
        fill='tozeroy',
        hoveron='points',
        hovertemplate = f'<b>{name}</b><br>'
            + 'CR delta %{x:.0f}<br>'
            + 'Monsters %{y:.1%}'
            + '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-sp-delta-by-ability-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-sp-delta-by-ability-small')
    pass

In [172]:
# plot the number of monsters by the type of change
import math

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='relative',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters changed (%)',
            #range=[0,100],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=2),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

stats = ['HP','AC','SB','DPR','AB','DC']

df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['eXP'] + stats, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['eXP']['MM'].isna() == False]
dfMP = dfMP[dfMP['eXP']['MM (2024)'].isna() == False]
m_count = dfMP.shape[0]

data = {}
for stat in stats:
    monsters = dfMP['Monster'].to_list()
    vals_24 = dfMP[stat]['MM (2024)'].to_list()
    vals_14 = dfMP[stat]['MM'].to_list()
    delta = []
    for v2, v1, m in zip(vals_24, vals_14, monsters):
        if math.isnan(v2) and math.isnan(v1):
            delta.append(False)
        elif math.isnan(v2):
            delta.append(False)
            #print(m, v2, v1, delta[-1])
        elif math.isnan(v1):
            delta.append(True)
            #print(m, v2, v1, delta[-1])
        else:
            delta.append(v2 > v1)
    data[stat] = np.sum(delta)
    #data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) > 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='higher',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))

data = {}
for stat in stats:
    monsters = dfMP['Monster'].to_list()
    vals_24 = dfMP[stat]['MM (2024)'].to_list()
    vals_14 = dfMP[stat]['MM'].to_list()
    delta = []
    for v2, v1, m in zip(vals_24, vals_14, monsters):
        if math.isnan(v2) and math.isnan(v1):
            delta.append(False)
        elif math.isnan(v2):
            delta.append(True)
        elif math.isnan(v1):
            delta.append(False)
        else:
            delta.append(v2 < v1)
        #print(v2, v1, delta[-1])
    data[stat] = np.sum(delta)
    #data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) < 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(-100/m_count),
    name='lower',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-stat-changes-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-stat-changes-small')

In [173]:
# plot the number of monsters by the type of change
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='relative',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters changed (%)',
            #range=[0,100],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=2),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

stats = ['adj HP','adj AC','adj SB','adj DPR','adj AB','adj DC']

df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['eXP'] + stats, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['eXP']['MM'].isna() == False]
dfMP = dfMP[dfMP['eXP']['MM (2024)'].isna() == False]
m_count = dfMP.shape[0]
print(m_count)

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) > 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='higher',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) < 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(-100/m_count),
    name='lower',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-stat-changes-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-stat-changes-small')

263


In [729]:
# plot the number of monsters by the type of change
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=55, b=85, t=20, pad=4),
        barmode='relative',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters changed (%)',
            #range=[0,100],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP'] + CONDITION_COLUMNS, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]
m_count = dfMP.shape[0]
print(m_count)

stats = CONDITION_COLUMNS
data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) > 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='higher',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

stats = CONDITION_COLUMNS
data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) < 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(-100/m_count),
    name='lower',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-stat-changes-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-stat-changes-small')

263
{'Blinded': 3, 'Charmed': 7, 'Deafened': 4, 'Exhaustion': 6, 'Frightened': 10, 'Grappled': 3, 'Incapacitated': 0, 'Invisible': 0, 'Paralyzed': 2, 'Petrified': 4, 'Poisoned': 2, 'Prone': 0, 'Restrained': 4, 'Stunned': 3, 'Unconscious': 4}
{'Blinded': 5, 'Charmed': 0, 'Deafened': 0, 'Exhaustion': 0, 'Frightened': 0, 'Grappled': 0, 'Incapacitated': 0, 'Invisible': 0, 'Paralyzed': 0, 'Petrified': 0, 'Poisoned': 0, 'Prone': 0, 'Restrained': 0, 'Stunned': 0, 'Unconscious': 0}


In [ ]:
def get_mitigation_type(val):
    if val == 0:
        mt = 'I'
    elif val == 2:
        mt = 'V'
    elif val == 1:
        mt = 'N'
    elif val == 0.5:
        mt = 'R'
    elif val < 0.5:
        mt = 'I-N'
    elif val < 1.0:
        mt = 'R-N'
    else:
        mt = '?'
    return mt

{
    'R-N -> R': 10,
    'R-N -> N': 43,
    'I-N -> R': 4,
    'I-N -> N': 11,
    'N -> R': 1,
    'N -> I': 0,
    'R -> N': 0,
    'R -> I': 0,
    'I -> N': 0,
    'I -> R': 0,
}


df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP','adj HP'] + DAMAGE_COLUMNS, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]

cm = 0
for dt in ['Bludgeoning','Slashing','Piercing']:
    cv = {}
    mt14 = dfMP.apply(lambda r: get_mitigation_type(r[dt]['MM']), axis=1).to_list()
    mt24 = dfMP.apply(lambda r: get_mitigation_type(r[dt]['MM (2024)']), axis=1).to_list()

    cmt = []
    for m1, m2 in zip(mt14, mt24):
        cv[f'{m1} -> {m2}'] = cv.get(f'{m1} -> {m2}', 0) + 1
        if f'{m1} -> {m2}' in ['N -> N','R -> R','I -> I','V -> V']:
            cmt.append(0)
        else:
            cmt.append(1)
    cm += np.array(cmt)

    print(dt)
    for k, v in cv.items():
        if k in ['N -> N','R -> R','I -> I','V -> V']: continue
        print(f'  {k}: {v}')

print(f'total monsters changed: {np.count_nonzero(cm)}/{len(cm)}')


monster = dfMP['Monster'].to_list()
cr = dfMP['CR'].to_list()
hp14 = dfMP['HP']['MM'].to_list()
hp24 = dfMP['HP']['MM (2024)'].to_list()
ahp14 = dfMP['adj HP']['MM'].to_list()
ahp24 = dfMP['adj HP']['MM (2024)'].to_list()
changed_monsters = {
    'CR': [],
    'Monster': [],
    'HP ratio': [],
    'adj HP ratio': [],
}
for i in range(len(cm)):
    if cm[i] > 0:
        changed_monsters['CR'].append(cr[i])
        changed_monsters['Monster'].append(monster[i])
        changed_monsters['HP ratio'].append(hp24[i]/hp14[i])
        changed_monsters['adj HP ratio'].append(ahp24[i]/ahp14[i])

dfCM = pd.DataFrame(changed_monsters)
print('average HP change:', np.mean(dfCM['HP ratio']))
print('average adj HP change:', np.mean(dfCM['adj HP ratio']))
#dfCM

Bludgeoning
  R-N -> R: 10
  R-N -> N: 43
  I-N -> R: 4
  I-N -> N: 11
  N -> R: 1
Slashing
  R-N -> R: 10
  R-N -> N: 44
  I-N -> R: 4
  I-N -> N: 11
  N -> R: 1
Piercing
  R-N -> R: 10
  R-N -> N: 44
  I-N -> R: 4
  R -> N: 1
  I-N -> N: 10
  N -> R: 1
total monsters changed: 71/263
average HP change: 1.2847376010230849
average adj HP change: 1.0888545180273053


In [ ]:
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        barmode='stack',
        yaxis=dict(
            title_text='monsters',
            range=[0,120],
            tick0=0, dtick=20,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.01, y=0.99,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

def get_mitigation_type(val):
    if val == 0:
        mt = 'I'
    elif val == 2:
        mt = 'V'
    elif val == 1:
        mt = 'N'
    elif val == 0.5:
        mt = 'R'
    elif val < 0.5:
        mt = 'I-N'
    elif val < 1.0:
        mt = 'R-N'
    else:
        mt = '?'
    return mt

conv_types = {
    'R -> I': '#1f77b4',
    'R -> N': 0,
    'R-N -> R': 10,
    'R-N -> N': 43,
    'I -> R': 0,
    'I -> N': 0,
    'I-N -> R': 4,
    'I-N -> N': 11,
    'N -> I': 0,
    'N -> R': 1,
}


df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP','adj HP'] + DAMAGE_COLUMNS, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]

cv_dict = {}
columns = DAMAGE_COLUMNS[:3]
for dt in columns:
    cv = {}
    mt14 = dfMP.apply(lambda r: get_mitigation_type(r[dt]['MM']), axis=1).to_list()
    mt24 = dfMP.apply(lambda r: get_mitigation_type(r[dt]['MM (2024)']), axis=1).to_list()

    cv_dict[dt] = {}
    for m1, m2 in zip(mt14, mt24):
        ct = f'{m1} -> {m2}'
        cv_dict[dt][ct] = cv_dict[dt].get(ct, 0) + 1


for ct in conv_types:
    y = [cv_dict[column].get(ct, 0) for column in columns]
    if sum(y) == 0: continue
    #y = np.array(y)
    #text = [f'{ty} out of {tt}' for ty, tt in zip(y, t)]
    fig.add_trace(go.Bar(
        x=columns,
        y=y,
        name=ct,
        #text=text,
        textposition='none',
        #marker_color=color,
        #hovertemplate=
        #    '<b>' + name + ' to %{x}</b><br>'+
        #    '%{y:.1f}% (%{text})'+
        #    '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-physical-modifier-types-by-damage-large')
    #tfb.save_fig_html(fig, format='small', name='fig-physical-modifier-types-by-damage-small', style='aspect-ratio: 600/500;')
    pass

In [49]:



def get_mitigation_type(val):
    if val == 0:
        mt = 'I'
    elif val == 2:
        mt = 'V'
    elif val == 1:
        mt = 'N'
    elif val == 0.5:
        mt = 'R'
    elif val < 0.5:
        mt = 'I-N'
    elif val < 1.0:
        mt = 'R-N'
    else:
        mt = '?'
    return mt

conv_types = {
    'R -> I': {'offsetgroup': 'R'},
    'R -> N': {'offsetgroup': 'R'},
    'R-N -> R': {'offsetgroup': 'R'},
    'R-N -> N': {'offsetgroup': 'R'},
    'I -> R': {'offsetgroup': 'I'},
    'I -> N': {'offsetgroup': 'I'},
    'I-N -> R': {'offsetgroup': 'I'},
    'I-N -> N': {'offsetgroup': 'I'},
    'N -> I': {'offsetgroup': 'N'},
    'N -> R': {'offsetgroup': 'N'},
}


df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP','adj HP'] + DAMAGE_COLUMNS, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]

cv_dict = {}
columns = DAMAGE_COLUMNS[3:]
for dt in columns:
    cv = {}
    mt14 = dfMP.apply(lambda r: get_mitigation_type(r[dt]['MM']), axis=1).to_list()
    mt24 = dfMP.apply(lambda r: get_mitigation_type(r[dt]['MM (2024)']), axis=1).to_list()

    cv_dict[dt] = {}
    for m1, m2 in zip(mt14, mt24):
        ct = f'{m1} -> {m2}'
        cv_dict[dt][ct] = cv_dict[dt].get(ct, 0) + 1

data = []
for ct in conv_types:
    y = [cv_dict[column].get(ct, 0) for column in columns]
    if sum(y) == 0: continue
    #y = np.array(y)
    #text = [f'{ty} out of {tt}' for ty, tt in zip(y, t)]
    data.append(go.Bar(
        x=columns,
        y=y,
        name=ct,
        #text=text,
        #textposition='none',
        offsetgroup=conv_types[ct]['offsetgroup'],
        #marker_color=color,
        #hovertemplate=
        #    '<b>' + name + ' to %{x}</b><br>'+
        #    '%{y:.1f}% (%{text})'+
        #    '<extra></extra>'
    ))

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        barmode='stack',
        yaxis=dict(
            title_text='monsters',
            range=[0,20],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.01, y=0.99,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    ),
    data=data,
)

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name='fig-physical-modifier-types-by-damage-large')
    #tfb.save_fig_html(fig, format='small', name='fig-physical-modifier-types-by-damage-small', style='aspect-ratio: 600/500;')
    pass

In [ ]:
def get_mitigation_type(val):
    if val == 0:
        mt = 'I'
    elif val == 2:
        mt = 'V'
    elif val == 1:
        mt = 'N'
    elif val == 0.5:
        mt = 'R'
    elif val < 0.5:
        mt = 'I-N'
    elif val < 1.0:
        mt = 'R-N'
    else:
        mt = '?'
    return mt

df1 = df0[df0['Book'].isin(['MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]

for dt in DAMAGE_COLUMNS:
    cv = {}
    mt = df1.apply(lambda r: get_mitigation_type(r[dt]), axis=1).to_list()

    for m in mt:
        cv[m] = cv.get(m, 0) + 1
    print(dt)
    for k, v in cv.items():
        print(f'  {k}: {v}')

Bludgeoning
  N: 375
  R: 28
  V: 2
Piercing
  N: 377
  R: 28
Slashing
  N: 377
  R: 26
  I: 2
Acid
  N: 365
  I: 22
  R: 18
Cold
  N: 316
  I: 27
  R: 59
  V: 3
Fire
  N: 308
  I: 48
  R: 43
  V: 6
Force
  N: 405
Lightning
  N: 349
  I: 24
  R: 32
Necrotic
  N: 351
  R: 24
  I: 30
Poison
  N: 294
  I: 107
  R: 4
Psychic
  N: 374
  R: 15
  I: 16
Radiant
  N: 387
  R: 14
  I: 3
  V: 1
Thunder
  N: 390
  I: 4
  R: 10
  V: 1


In [733]:
# plot the number of monsters by the type of change
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=35, b=75, t=20, pad=4),
        barmode='relative',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters changed (%)',
            #range=[0,100],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='right', yanchor='bottom',
            x=1.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP'] + DAMAGE_COLUMNS, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]
m_count = dfMP.shape[0]
print(m_count)

stats = DAMAGE_COLUMNS

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) < 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='higher',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) > 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(-100/m_count),
    name='lower',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-stat-changes-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-stat-changes-small')

263
{'Bludgeoning': 11, 'Piercing': 11, 'Slashing': 11, 'Acid': 2, 'Cold': 4, 'Fire': 1, 'Force': 0, 'Lightning': 0, 'Necrotic': 7, 'Poison': 1, 'Psychic': 5, 'Radiant': 5, 'Thunder': 1}
{'Bludgeoning': 58, 'Piercing': 59, 'Slashing': 59, 'Acid': 1, 'Cold': 4, 'Fire': 1, 'Force': 2, 'Lightning': 6, 'Necrotic': 2, 'Poison': 1, 'Psychic': 0, 'Radiant': 1, 'Thunder': 2}


In [734]:
# plot the number of monsters by the type of change
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='relative',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters changed (%)',
            #range=[0,100],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP'] + ABILITY_MOD_COLUMNS, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]
m_count = dfMP.shape[0]
print(m_count)

stats = ABILITY_MOD_COLUMNS

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) > 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='higher',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) < 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(-100/m_count),
    name='lower',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-stat-changes-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-stat-changes-small')

263
{'Str Mod': 11, 'Dex Mod': 13, 'Con Mod': 3, 'Int Mod': 3, 'Wis Mod': 8, 'Cha Mod': 25}
{'Str Mod': 2, 'Dex Mod': 0, 'Con Mod': 2, 'Int Mod': 1, 'Wis Mod': 0, 'Cha Mod': 3}


In [735]:
# plot the number of monsters by the type of change
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='relative',
        #xaxis=dict(title_text='change'),
        yaxis=dict(
            title_text='monsters changed (%)',
            #range=[0,100],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

df1 = df0[df0['Book'].isin(['MM','MM (2024)']) & df0['Legendary'].isin(['N','L','LL'])]
dfMP = pd.pivot_table(df1, 
    values=['HP'] + SAVE_BONUS_COLUMNS, 
    index=['Monster', 'CR'], columns=['Book']).reset_index()
dfMP = dfMP[dfMP['HP']['MM'].isna() == False]
dfMP = dfMP[dfMP['HP']['MM (2024)'].isna() == False]
m_count = dfMP.shape[0]
print(m_count)

stats = SAVE_BONUS_COLUMNS

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) > 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(100/m_count),
    name='higher',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

data = {}
for stat in stats:
    data[stat] = np.sum((dfMP[stat]['MM (2024)'] - dfMP[stat]['MM']) < 0)

fig.add_trace(go.Bar(
    x=np.array(list(data.keys())),
    y=np.array(list(data.values()))*(-100/m_count),
    name='lower',
    hovertemplate = ''
        + 'Raw %{y:,.1f}%'
        + '<extra></extra>'
))
print(data)

# show figure
fig.update_layout(width=600, height=450, hovermode='x unified')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-stat-changes-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-stat-changes-small')

263
{'Str Save': 16, 'Dex Save': 20, 'Con Save': 10, 'Int Save': 3, 'Wis Save': 12, 'Cha Save': 8}
{'Str Save': 5, 'Dex Save': 1, 'Con Save': 46, 'Int Save': 12, 'Wis Save': 4, 'Cha Save': 62}
